In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv("diabetes_binary_classification_data.csv")

In [15]:
# Define features (X) and target (y)
X = data.drop('Diabetes_binary', axis=1)
y = data['Diabetes_binary']

# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 15% validation, 15% test

# Create preprocessing pipelines for numeric and categorical features
numeric_features = ['BMI', 'PhysHlth', 'Age']  # Non-binary variables
categorical_features = ['GenHlth', 'HighBP', 'HighChol', 'HeartDiseaseorAttack', 'DiffWalk']  # Binary variables

# Create preprocessing for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocess training data (numerical and categorical features)
X_train_processed = preprocessor.fit_transform(X_train)

# Preprocess validation and test data
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

In [16]:
# Define models
log_reg = LogisticRegression(class_weight='balanced', max_iter=2000, random_state=37)
decision_tree = DecisionTreeClassifier(class_weight='balanced', max_depth=50, random_state=74)
random_forest = RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=50, random_state=20)

# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('decision_tree', decision_tree),
    ('random_forest', random_forest)],
    voting='hard')  # 'hard' for majority voting

# Fit models on the processed training data
log_reg.fit(X_train_processed, y_train)
decision_tree.fit(X_train_processed, y_train)
random_forest.fit(X_train_processed, y_train)
voting_clf.fit(X_train_processed, y_train)

# Make predictions on the validation set
log_reg_val_pred = log_reg.predict(X_val_processed)
decision_tree_val_pred = decision_tree.predict(X_val_processed)
random_forest_val_pred = random_forest.predict(X_val_processed)
voting_val_pred = voting_clf.predict(X_val_processed)

# Make predictions on the test set
log_reg_test_pred = log_reg.predict(X_test_processed)
decision_tree_test_pred = decision_tree.predict(X_test_processed)
random_forest_test_pred = random_forest.predict(X_test_processed)
voting_test_pred = voting_clf.predict(X_test_processed)

# Make predictions on the test set
log_reg_test_pred = log_reg.predict(X_test_processed)
decision_tree_test_pred = decision_tree.predict(X_test_processed)
random_forest_test_pred = random_forest.predict(X_test_processed)
voting_test_pred = voting_clf.predict(X_test_processed)

# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('log_reg', log_reg),
    ('decision_tree', decision_tree),
    ('random_forest', random_forest)],
    voting='hard')  # 'hard' for majority voting

In [17]:
# Evaluate models on validation data and print results
for model_name, predictions in zip(
    ["Logistic Regression", "Decision Tree", "Random Forest", "Voting Classifier"],
    [log_reg_test_pred, decision_tree_test_pred, random_forest_test_pred, voting_test_pred]
):
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name}:")
    print(classification_report(y_test, predictions))
    print('confusion matrix:')
    print(confusion_matrix(y_test, predictions, labels=[0,1]))
    print('-'*64)
    print('')

Logistic Regression:
              precision    recall  f1-score   support

         0.0       0.95      0.72      0.82     32858
         1.0       0.30      0.77      0.43      5194

    accuracy                           0.72     38052
   macro avg       0.63      0.74      0.62     38052
weighted avg       0.86      0.72      0.76     38052

confusion matrix:
[[23570  9288]
 [ 1220  3974]]
----------------------------------------------------------------

Decision Tree:
              precision    recall  f1-score   support

         0.0       0.91      0.78      0.84     32858
         1.0       0.28      0.53      0.36      5194

    accuracy                           0.75     38052
   macro avg       0.59      0.66      0.60     38052
weighted avg       0.83      0.75      0.78     38052

confusion matrix:
[[25633  7225]
 [ 2437  2757]]
----------------------------------------------------------------

Random Forest:
              precision    recall  f1-score   support

         0